In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']])

In [12]:
community = 'san_jose'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Heat Pump Water Heater                                              4004.7
Basic Enclosure                                                     8279.2
Mininum Efficiency Heat Pump with Electric Heat Backup              9969.7
Enhanced Enclosure                                                 10337.5
Mininum Efficiency Whole Home Electrification                      11373.5
Mininum Efficiency Heat Pump with Existing Heat Backup             11644.7
High Efficiency Heat Pump with Electric Heat Backup                27872.2
High Efficiency Whole Home Electrification                         36277.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        36555.0
Enhanced Enc

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Mininum Efficiency Heat Pump with Existing Heat Backup             82906.438235
Mininum Efficiency Heat Pump with Electric Heat Backup             83151.000000
Electric Clothes Dryer                                             18029.580486
Electric Cooking                                                   59015.195250
Induction Cooking                                                  81474.190316
Heat Pump Clothes Dryer                                            62477.933271
High Efficiency Heat Pump with Electric Heat Backup                83151.000000
Mininum Efficiency Whole Home Electrification                      79792.351765
Heat Pump Water Heater                                             82682.256618
Basic Enclosure                                                    77857.765018
Enhanced Enclosure                                                 78981.181432
High Efficiency Whole Home Electrification                         83151.000000
Basic Enclosure + HPWH + Hi

In [18]:
metric = "pct_saving_energy.total_%"
print("Community summary:")
get_summary(community_results, metric, lower_percentile=0.25, upper_percentile=0.75)

Community summary:


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1605,83151.000000,43.399736,33.824410,53.610000
Basic Enclosure + High Efficiency Whole Home Electrification,1605,83151.000000,43.233000,33.305824,53.384847
High Efficiency Whole Home Electrification,1605,83151.000000,40.399159,31.567398,50.750622
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1605,83151.000000,38.368719,27.780424,50.132244
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1605,83151.000000,38.202125,27.593171,50.024528
Mininum Efficiency Whole Home Electrification,1583,79792.351765,30.218578,19.133915,42.118302
Heat Pump Water Heater,1599,82682.256618,22.775241,13.874127,30.742321
Enhanced Enclosure,1560,78981.181432,12.622413,1.633557,20.477467
Basic Enclosure,1550,77857.765018,11.595405,1.562636,18.479251


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Multifamily with 5+ units, 1940-1979, all income 

Segment 2. Occupied, Multifamily with 5+ units, 1980+, all income 

Segment 3. Occupied, Single family detached 1940-1979, all income 

Segment 4. Occupied, Single family attached 1940-1979, all income 


In [19]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Multi-Family with 5+ Units']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1980s', '1990s', '2000s', '2010s']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Single-Family Attached']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

In [20]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    display(get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75))

Summary for segment: segment1: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,143,9101.533910,41.168750,32.124486,50.376584
Basic Enclosure + High Efficiency Whole Home Electrification,143,9101.533910,40.780048,31.439008,50.376584
High Efficiency Whole Home Electrification,143,9101.533910,39.657092,30.390624,48.984562
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,143,9101.533910,35.317116,25.954574,44.841715
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,143,9101.533910,34.928364,25.919653,44.754023
Mininum Efficiency Whole Home Electrification,139,8636.866557,31.883543,24.797307,40.304756
Heat Pump Water Heater,142,8979.253028,25.359542,18.617292,32.067861
Enhanced Enclosure,142,9087.147924,8.057093,1.855710,13.225198
Basic Enclosure,139,8940.410865,6.999158,1.800963,11.697476


Summary for segment: segment2: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,104,12128.040241,37.538687,31.009697,46.500000
Basic Enclosure + High Efficiency Whole Home Electrification,104,12128.040241,37.435005,30.865769,46.109917
High Efficiency Whole Home Electrification,104,12128.040241,37.053599,28.741818,45.633802
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,104,12128.040241,32.279029,24.177727,40.282727
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,104,12128.040241,32.175280,24.177727,39.987636
Mininum Efficiency Whole Home Electrification,101,11516.635829,31.610940,25.033091,41.909177
Heat Pump Water Heater,104,12128.040241,29.824485,24.284295,37.579182
Electric Cooking,66,7857.525997,5.923723,4.820000,7.000255
Induction Cooking,100,11700.057152,4.486117,0.626983,6.840710


Summary for segment: segment3: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,324,18303.007957,49.975292,39.681553,60.658954
Basic Enclosure + High Efficiency Whole Home Electrification,324,18303.007957,49.821133,39.241196,60.670000
High Efficiency Whole Home Electrification,324,18303.007957,45.651783,35.802389,55.431567
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,324,18303.007957,45.173167,33.820865,57.060065
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,324,18303.007957,45.019513,34.203116,56.983462
Mininum Efficiency Whole Home Electrification,323,18058.446193,34.106703,23.012199,44.849805
Enhanced Enclosure,324,18303.007957,20.528597,8.391694,31.123987
Heat Pump Water Heater,324,18303.007957,20.145229,14.064240,26.608162
Basic Enclosure,323,18241.867516,18.920226,7.640597,28.524199


Summary for segment: segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,24,3668.426471,44.665844,34.319500,57.010000
Basic Enclosure + High Efficiency Whole Home Electrification,24,3668.426471,44.580378,34.319500,57.010000
High Efficiency Whole Home Electrification,24,3668.426471,41.446444,31.344889,52.550000
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,24,3668.426471,38.238422,27.175000,50.605000
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,24,3668.426471,38.153622,27.171500,50.605000
Mininum Efficiency Whole Home Electrification,22,3342.344118,28.989195,19.808000,41.991944
Heat Pump Water Heater,24,3668.426471,16.745333,10.649667,22.600000
Enhanced Enclosure,24,3668.426471,13.592511,2.740000,23.132667
Basic Enclosure,24,3668.426471,12.482378,2.100000,21.987000


## 2.1 Upgrade Cost Per Dwelling Unit 

In [21]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Basic Enclosure                                                     2826.5
Heat Pump Water Heater                                              3761.0
Enhanced Enclosure                                                  3997.3
Mininum Efficiency Whole Home Electrification                       5676.9
Mininum Efficiency Heat Pump with Electric Heat Backup              5732.1
Mininum Efficiency Heat Pump with Existing Heat Backup              7571.1
High Efficiency Heat Pump with Electric Heat Backup                19346.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        24565.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25493.1
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                     1384.6
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Enhanced Enclosure                                                  1724.4
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Heat Pump Water Heater                                              3777.0
Mininum Efficiency Whole Home Electrification                       4799.5
Mininum Efficiency Heat Pump with Electric Heat Backup              6731.7
Mininum Efficiency Heat Pump with Existing Heat Backup              7788.3
High Efficiency Heat Pump with Electric Heat Backup                21984.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26568.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     26817.0
High Efficie

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Heat Pump Water Heater                                              4138.8
Mininum Efficiency Heat Pump with Electric Heat Backup             12093.6
Basic Enclosure                                                    13312.2
Mininum Efficiency Heat Pump with Existing Heat Backup             14351.9
Mininum Efficiency Whole Home Electrification                      15051.7
Enhanced Enclosure                                                 16949.8
High Efficiency Heat Pump with Electric Heat Backup                32312.5
High Efficiency Whole Home Electrification                         41442.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44172.4
Enhanced Enc

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Heat Pump Water Heater                                              3761.0
Basic Enclosure                                                     6315.1
Enhanced Enclosure                                                  7642.6
Mininum Efficiency Heat Pump with Electric Heat Backup              8728.2
Mininum Efficiency Heat Pump with Existing Heat Backup             10367.6
Mininum Efficiency Whole Home Electrification                      10508.6
High Efficiency Heat Pump with Electric Heat Backup                26072.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        33975.3
High Efficiency Whole Home Electrification                         34707.2
Enhanced Enc

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [22]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Heat Pump Water Heater                                             0.02611
Basic Enclosure                                                    0.02254
Enhanced Enclosure                                                 0.01905
Mininum Efficiency Whole Home Electrification                      0.01500
Heat Pump Clothes Dryer                                            0.01115
Basic Enclosure + High Efficiency Whole Home Electrification       0.00722
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00722
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00718
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00717
High Efficiency Whole Home Electrification                         0.00714
Induction Cooking                                                  0.00330
Electric Cooking                                                   0.00229
High Efficiency Heat Pump with Electric Heat Backup                0.00063
Mininum Effi

segment2


upgrade_name
Heat Pump Water Heater                                             0.04129
Mininum Efficiency Whole Home Electrification                      0.02618
Heat Pump Clothes Dryer                                            0.01049
Enhanced Enclosure                                                 0.00758
Basic Enclosure                                                    0.00689
High Efficiency Whole Home Electrification                         0.00687
Basic Enclosure + High Efficiency Whole Home Electrification       0.00685
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00683
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00683
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00681
Electric Cooking                                                   0.00396
Induction Cooking                                                  0.00380
High Efficiency Heat Pump with Electric Heat Backup               -0.00092
Electric Clo

segment3


upgrade_name
Heat Pump Water Heater                                             0.05104
Basic Enclosure                                                    0.02697
Enhanced Enclosure                                                 0.02285
Heat Pump Clothes Dryer                                            0.01632
Mininum Efficiency Whole Home Electrification                      0.01626
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01453
Basic Enclosure + High Efficiency Whole Home Electrification       0.01420
High Efficiency Whole Home Electrification                         0.01387
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01377
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01354
High Efficiency Heat Pump with Electric Heat Backup                0.00764
Induction Cooking                                                  0.00381
Electric Cooking                                                   0.00253
Mininum Effi

segment4


upgrade_name
Heat Pump Water Heater                                             0.02902
Basic Enclosure                                                    0.02208
Enhanced Enclosure                                                 0.01991
Heat Pump Clothes Dryer                                            0.01331
Mininum Efficiency Whole Home Electrification                      0.01128
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01002
Basic Enclosure + High Efficiency Whole Home Electrification       0.00998
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00983
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00982
High Efficiency Whole Home Electrification                         0.00954
High Efficiency Heat Pump with Electric Heat Backup                0.00368
Induction Cooking                                                  0.00367
Electric Cooking                                                   0.00200
Mininum Effi

 ### Annual Energy Savings Per Dollar Invested

In [23]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Heat Pump Water Heater                                             0.00179
Mininum Efficiency Whole Home Electrification                      0.00155
Electric Cooking                                                   0.00082
Basic Enclosure                                                    0.00068
Enhanced Enclosure                                                 0.00057
Induction Cooking                                                  0.00047
High Efficiency Whole Home Electrification                         0.00041
Basic Enclosure + High Efficiency Whole Home Electrification       0.00040
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00039
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00039
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00038
Heat Pump Clothes Dryer                                            0.00031
Electric Clothes Dryer                                             0.00024
High Efficie

segment2


upgrade_name
Heat Pump Water Heater                                             0.00212
Mininum Efficiency Whole Home Electrification                      0.00183
Electric Cooking                                                   0.00086
Induction Cooking                                                  0.00043
High Efficiency Whole Home Electrification                         0.00035
Basic Enclosure + High Efficiency Whole Home Electrification       0.00034
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00034
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00034
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00034
Enhanced Enclosure                                                 0.00027
Heat Pump Clothes Dryer                                            0.00025
Basic Enclosure                                                    0.00023
Electric Clothes Dryer                                             0.00017
High Efficie

segment3


upgrade_name
Heat Pump Water Heater                                             0.00310
Mininum Efficiency Whole Home Electrification                      0.00167
Electric Cooking                                                   0.00109
Basic Enclosure                                                    0.00106
Enhanced Enclosure                                                 0.00090
High Efficiency Whole Home Electrification                         0.00078
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00073
Basic Enclosure + High Efficiency Whole Home Electrification       0.00072
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00071
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00069
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00068
Heat Pump Clothes Dryer                                            0.00059
Induction Cooking                                                  0.00059
Mininum Effi

segment4


upgrade_name
Heat Pump Water Heater                                             0.00152
Mininum Efficiency Whole Home Electrification                      0.00119
Basic Enclosure                                                    0.00085
Electric Cooking                                                   0.00083
Enhanced Enclosure                                                 0.00077
Induction Cooking                                                  0.00051
High Efficiency Whole Home Electrification                         0.00048
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00046
Basic Enclosure + High Efficiency Whole Home Electrification       0.00046
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00045
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00045
Heat Pump Clothes Dryer                                            0.00038
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00030
Electric Clo

 ### Annual Emission Savings Per Dollar Invested

In [24]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.13944
Heat Pump Water Heater                                             0.13895
Electric Cooking                                                   0.08780
Electric Clothes Dryer                                             0.05427
Induction Cooking                                                  0.04589
Basic Enclosure                                                    0.03193
High Efficiency Whole Home Electrification                         0.03097
Basic Enclosure + High Efficiency Whole Home Electrification       0.02958
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02872
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02859
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02765
Enhanced Enclosure                                                 0.02600
Heat Pump Clothes Dryer                                            0.01082
Mininum Effi

segment2


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.14860
Heat Pump Water Heater                                             0.14650
Electric Cooking                                                   0.09147
Induction Cooking                                                  0.04129
Electric Clothes Dryer                                             0.03722
High Efficiency Whole Home Electrification                         0.02477
Basic Enclosure + High Efficiency Whole Home Electrification       0.02419
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02400
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02322
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02302
Enhanced Enclosure                                                 0.01444
Basic Enclosure                                                    0.01188
Heat Pump Clothes Dryer                                            0.00632
High Efficie

segment3


upgrade_name
Heat Pump Water Heater                                             0.23599
Mininum Efficiency Whole Home Electrification                      0.15098
Electric Cooking                                                   0.11582
Electric Clothes Dryer                                             0.09207
Mininum Efficiency Heat Pump with Electric Heat Backup             0.08031
Basic Enclosure                                                    0.06279
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06210
High Efficiency Whole Home Electrification                         0.05891
Induction Cooking                                                  0.05733
Enhanced Enclosure                                                 0.05351
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.05173
Basic Enclosure + High Efficiency Whole Home Electrification       0.05101
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04873
Enhanced Enc

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.10892
Heat Pump Water Heater                                             0.10782
Electric Cooking                                                   0.09018
Induction Cooking                                                  0.05006
Basic Enclosure                                                    0.04984
Electric Clothes Dryer                                             0.04892
Enhanced Enclosure                                                 0.04602
Mininum Efficiency Heat Pump with Electric Heat Backup             0.04136
High Efficiency Whole Home Electrification                         0.03381
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03122
Basic Enclosure + High Efficiency Whole Home Electrification       0.03098
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03072
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03035
Enhanced Enc

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [25]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1891074.8
Basic Enclosure + High Efficiency Whole Home Electrification       1841076.8
High Efficiency Whole Home Electrification                         1721768.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1664464.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1614466.3
Heat Pump Water Heater                                              881858.2
Mininum Efficiency Whole Home Electrification                       735335.7
Enhanced Enclosure                                                  691944.3
Basic Enclosure                                                     569553.5
High Efficiency Heat Pump with Electric Heat Backup                 110529.0
Heat Pump Clothes Dryer                                              91600.7
Induction Cooking                                                    76092.1
Electric Cooking                                               

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2506745.0
Basic Enclosure + High Efficiency Whole Home Electrification       2492783.1
High Efficiency Whole Home Electrification                         2433782.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2215732.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2201770.8
Heat Pump Water Heater                                             1891196.0
Mininum Efficiency Whole Home Electrification                      1447070.2
Enhanced Enclosure                                                  124215.4
Heat Pump Clothes Dryer                                             121107.1
Induction Cooking                                                   115211.6
Basic Enclosure                                                      89074.2
Electric Cooking                                                     52854.7
Electric Clothes Dryer                                         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    12873400.8
Basic Enclosure + High Efficiency Whole Home Electrification       12781497.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     11837980.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11746077.3
High Efficiency Whole Home Electrification                         10521089.3
Enhanced Enclosure                                                  7087858.2
Basic Enclosure                                                     6549768.7
High Efficiency Heat Pump with Electric Heat Backup                 4516692.9
Mininum Efficiency Whole Home Electrification                       4420256.7
Heat Pump Water Heater                                              3866071.8
Heat Pump Clothes Dryer                                              729078.0
Induction Cooking                                                    177628.9
Electric Cooking                                   

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1430600.5
Basic Enclosure + High Efficiency Whole Home Electrification       1422503.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1257134.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1249037.7
High Efficiency Whole Home Electrification                         1214703.6
Enhanced Enclosure                                                  558069.8
Basic Enclosure                                                     511427.4
Heat Pump Water Heater                                              400361.4
Mininum Efficiency Whole Home Electrification                       396274.7
High Efficiency Heat Pump with Electric Heat Backup                 351875.5
Heat Pump Clothes Dryer                                             114275.5
Induction Cooking                                                    34153.4
Electric Cooking                                               

### Annual Total Energy Savings

In [26]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    102371.7
Basic Enclosure + High Efficiency Whole Home Electrification       101224.4
High Efficiency Whole Home Electrification                          98486.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      89087.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         87940.2
Mininum Efficiency Whole Home Electrification                       75847.7
Heat Pump Water Heater                                              60428.4
Enhanced Enclosure                                                  20613.3
Basic Enclosure                                                     17267.9
Induction Cooking                                                   10757.9
Electric Cooking                                                     9632.0
High Efficiency Heat Pump with Electric Heat Backup                  7435.6
Heat Pump Clothes Dryer                                              2526.3

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    125381.6
Basic Enclosure + High Efficiency Whole Home Electrification       125061.2
High Efficiency Whole Home Electrification                         123707.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     109552.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        109232.3
Mininum Efficiency Whole Home Electrification                      101062.1
Heat Pump Water Heater                                              97110.2
Induction Cooking                                                   12898.3
Electric Cooking                                                    11467.3
Enhanced Enclosure                                                   4411.6
Basic Enclosure                                                      3001.3
Heat Pump Clothes Dryer                                              2930.6
Electric Clothes Dryer                                                 69.2

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    648475.6
Basic Enclosure + High Efficiency Whole Home Electrification       646366.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     594839.6
High Efficiency Whole Home Electrification                         594491.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        592730.2
Mininum Efficiency Whole Home Electrification                      454479.5
Enhanced Enclosure                                                 278757.0
High Efficiency Heat Pump with Electric Heat Backup                263399.1
Basic Enclosure                                                    256752.2
Heat Pump Water Heater                                             234546.8
Mininum Efficiency Heat Pump with Electric Heat Backup             156158.8
Mininum Efficiency Heat Pump with Existing Heat Backup             138055.8
Induction Cooking                                                   27337.4

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    65456.9
Basic Enclosure + High Efficiency Whole Home Electrification       65271.0
High Efficiency Whole Home Electrification                         60502.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     57494.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        57308.6
Mininum Efficiency Whole Home Electrification                      41719.8
Enhanced Enclosure                                                 21715.0
Heat Pump Water Heater                                             21011.3
High Efficiency Heat Pump with Electric Heat Backup                20282.0
Basic Enclosure                                                    19629.1
Mininum Efficiency Heat Pump with Electric Heat Backup              9591.4
Mininum Efficiency Heat Pump with Existing Heat Backup              7936.6
Induction Cooking                                                   4728.1
Electric Coo

### Annual Total Emission Savings

In [27]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    7563590.7
Basic Enclosure + High Efficiency Whole Home Electrification       7539600.6
High Efficiency Whole Home Electrification                         7468951.2
Mininum Efficiency Whole Home Electrification                      6837024.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6415669.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6391679.4
Heat Pump Water Heater                                             4692462.0
Induction Cooking                                                  1059090.4
Electric Cooking                                                   1034836.2
Enhanced Enclosure                                                  944306.3
Basic Enclosure                                                     806988.6
High Efficiency Heat Pump with Electric Heat Backup                 736910.9
Mininum Efficiency Heat Pump with Electric Heat Backup         

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    8811353.1
Basic Enclosure + High Efficiency Whole Home Electrification       8806716.3
High Efficiency Whole Home Electrification                         8775852.7
Mininum Efficiency Whole Home Electrification                      8213526.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     7486605.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        7481968.8
Heat Pump Water Heater                                             6710620.8
Induction Cooking                                                  1251799.9
Electric Cooking                                                   1221781.0
Enhanced Enclosure                                                  236593.2
Basic Enclosure                                                     153534.4
Heat Pump Clothes Dryer                                              72947.6
High Efficiency Heat Pump with Electric Heat Backup            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    45968413.5
Basic Enclosure + High Efficiency Whole Home Electrification       45901747.9
High Efficiency Whole Home Electrification                         44683078.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     41887164.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        41820499.2
Mininum Efficiency Whole Home Electrification                      41037317.8
High Efficiency Heat Pump with Electric Heat Backup                20197609.6
Heat Pump Water Heater                                             17877170.2
Mininum Efficiency Heat Pump with Electric Heat Backup             17775728.1
Enhanced Enclosure                                                 16600458.0
Mininum Efficiency Heat Pump with Existing Heat Backup             16311627.9
Basic Enclosure                                                    15248695.5
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4422425.8
Basic Enclosure + High Efficiency Whole Home Electrification       4415155.5
High Efficiency Whole Home Electrification                         4304594.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3835499.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3828228.9
Mininum Efficiency Whole Home Electrification                      3825742.7
High Efficiency Heat Pump with Electric Heat Backup                1572441.7
Heat Pump Water Heater                                             1487623.6
Mininum Efficiency Heat Pump with Electric Heat Backup             1324436.5
Enhanced Enclosure                                                 1290272.0
Mininum Efficiency Heat Pump with Existing Heat Backup             1187403.7
Basic Enclosure                                                    1154577.1
Induction Cooking                                              

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [28]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    207.8
Basic Enclosure + High Efficiency Whole Home Electrification       202.3
High Efficiency Whole Home Electrification                         189.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     182.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        177.4
Heat Pump Water Heater                                              98.2
Mininum Efficiency Whole Home Electrification                       85.1
Enhanced Enclosure                                                  76.1
Basic Enclosure                                                     63.7
Heat Pump Clothes Dryer                                             28.0
High Efficiency Heat Pump with Electric Heat Backup                 12.1
Induction Cooking                                                    8.5
Electric Cooking                                                     3.9
Electric Clothes Dryer                

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    206.7
Basic Enclosure + High Efficiency Whole Home Electrification       205.5
High Efficiency Whole Home Electrification                         200.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     182.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        181.5
Heat Pump Water Heater                                             155.9
Mininum Efficiency Whole Home Electrification                      125.7
Heat Pump Clothes Dryer                                             26.3
Enhanced Enclosure                                                  13.1
Induction Cooking                                                    9.8
Basic Enclosure                                                      9.5
Electric Cooking                                                     6.7
Electric Clothes Dryer                                             -14.6
High Efficiency Heat Pump with Electri

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    703.3
Basic Enclosure + High Efficiency Whole Home Electrification       698.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     646.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        641.8
High Efficiency Whole Home Electrification                         574.8
Enhanced Enclosure                                                 387.3
Basic Enclosure                                                    359.1
High Efficiency Heat Pump with Electric Heat Backup                246.8
Mininum Efficiency Whole Home Electrification                      244.8
Heat Pump Water Heater                                             211.2
Heat Pump Clothes Dryer                                             40.9
Induction Cooking                                                    9.9
Electric Cooking                                                     4.3
Mininum Efficiency Heat Pump with Elec

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    390.0
Basic Enclosure + High Efficiency Whole Home Electrification       387.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     342.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        340.5
High Efficiency Whole Home Electrification                         331.1
Enhanced Enclosure                                                 152.1
Basic Enclosure                                                    139.4
Mininum Efficiency Whole Home Electrification                      118.6
Heat Pump Water Heater                                             109.1
High Efficiency Heat Pump with Electric Heat Backup                 95.9
Heat Pump Clothes Dryer                                             33.4
Induction Cooking                                                    9.5
Electric Cooking                                                     3.4
Electric Clothes Dryer                

### Annual Energy Savings Per Dwelling Unit

In [29]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11.2
Basic Enclosure + High Efficiency Whole Home Electrification       11.1
High Efficiency Whole Home Electrification                         10.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      9.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         9.7
Mininum Efficiency Whole Home Electrification                       8.8
Heat Pump Water Heater                                              6.7
Enhanced Enclosure                                                  2.3
Basic Enclosure                                                     1.9
Electric Cooking                                                    1.4
Induction Cooking                                                   1.2
High Efficiency Heat Pump with Electric Heat Backup                 0.8
Heat Pump Clothes Dryer                                             0.8
Electric Clothes Dryer                             

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    10.3
Basic Enclosure + High Efficiency Whole Home Electrification       10.3
High Efficiency Whole Home Electrification                         10.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      9.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         9.0
Mininum Efficiency Whole Home Electrification                       8.8
Heat Pump Water Heater                                              8.0
Electric Cooking                                                    1.5
Induction Cooking                                                   1.1
Heat Pump Clothes Dryer                                             0.6
Enhanced Enclosure                                                  0.5
Basic Enclosure                                                     0.3
Electric Clothes Dryer                                              0.3
High Efficiency Heat Pump with Electric Heat Backup

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    35.4
Basic Enclosure + High Efficiency Whole Home Electrification       35.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32.5
High Efficiency Whole Home Electrification                         32.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32.4
Mininum Efficiency Whole Home Electrification                      25.2
Enhanced Enclosure                                                 15.2
High Efficiency Heat Pump with Electric Heat Backup                14.4
Basic Enclosure                                                    14.1
Heat Pump Water Heater                                             12.8
Mininum Efficiency Heat Pump with Electric Heat Backup              8.5
Mininum Efficiency Heat Pump with Existing Heat Backup              7.5
Electric Cooking                                                    1.8
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    17.8
Basic Enclosure + High Efficiency Whole Home Electrification       17.8
High Efficiency Whole Home Electrification                         16.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     15.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        15.6
Mininum Efficiency Whole Home Electrification                      12.5
Enhanced Enclosure                                                  5.9
Heat Pump Water Heater                                              5.7
High Efficiency Heat Pump with Electric Heat Backup                 5.5
Basic Enclosure                                                     5.4
Mininum Efficiency Heat Pump with Electric Heat Backup              2.6
Mininum Efficiency Heat Pump with Existing Heat Backup              2.2
Electric Cooking                                                    1.4
Induction Cooking                                  

### Annual Percentage Energy Savings Per Dwelling Unit

In [30]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    41.2
Basic Enclosure + High Efficiency Whole Home Electrification       40.8
High Efficiency Whole Home Electrification                         39.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        34.9
Mininum Efficiency Whole Home Electrification                      31.9
Heat Pump Water Heater                                             25.4
Enhanced Enclosure                                                  8.1
Basic Enclosure                                                     7.0
Electric Cooking                                                    5.5
Induction Cooking                                                   4.8
Heat Pump Clothes Dryer                                             3.2
High Efficiency Heat Pump with Electric Heat Backup                 1.7
Electric Clothes Dryer                             

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    37.5
Basic Enclosure + High Efficiency Whole Home Electrification       37.4
High Efficiency Whole Home Electrification                         37.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32.2
Mininum Efficiency Whole Home Electrification                      31.6
Heat Pump Water Heater                                             29.8
Electric Cooking                                                    5.9
Induction Cooking                                                   4.5
Heat Pump Clothes Dryer                                             2.5
Enhanced Enclosure                                                  1.5
Basic Enclosure                                                     1.1
Electric Clothes Dryer                                              1.0
High Efficiency Heat Pump with Electric Heat Backup

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    50.0
Basic Enclosure + High Efficiency Whole Home Electrification       49.8
High Efficiency Whole Home Electrification                         45.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     45.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        45.0
Mininum Efficiency Whole Home Electrification                      34.1
Enhanced Enclosure                                                 20.5
Heat Pump Water Heater                                             20.1
Basic Enclosure                                                    18.9
High Efficiency Heat Pump with Electric Heat Backup                17.1
Mininum Efficiency Heat Pump with Electric Heat Backup              7.6
Mininum Efficiency Heat Pump with Existing Heat Backup              6.3
Electric Cooking                                                    3.0
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    44.7
Basic Enclosure + High Efficiency Whole Home Electrification       44.6
High Efficiency Whole Home Electrification                         41.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     38.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        38.2
Mininum Efficiency Whole Home Electrification                      29.0
Heat Pump Water Heater                                             16.7
Enhanced Enclosure                                                 13.6
Basic Enclosure                                                    12.5
High Efficiency Heat Pump with Electric Heat Backup                11.3
Electric Cooking                                                    4.1
Induction Cooking                                                   3.9
Mininum Efficiency Heat Pump with Electric Heat Backup              3.5
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [31]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    831.0
Basic Enclosure + High Efficiency Whole Home Electrification       828.4
High Efficiency Whole Home Electrification                         820.6
Mininum Efficiency Whole Home Electrification                      791.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     704.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        702.3
Heat Pump Water Heater                                             522.6
Electric Cooking                                                   149.3
Induction Cooking                                                  118.9
Enhanced Enclosure                                                 103.9
Electric Clothes Dryer                                              91.8
Basic Enclosure                                                     90.3
High Efficiency Heat Pump with Electric Heat Backup                 81.0
Mininum Efficiency Heat Pump with Elec

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    726.5
Basic Enclosure + High Efficiency Whole Home Electrification       726.1
High Efficiency Whole Home Electrification                         723.6
Mininum Efficiency Whole Home Electrification                      713.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     617.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        616.9
Heat Pump Water Heater                                             553.3
Electric Cooking                                                   155.5
Induction Cooking                                                  107.0
Electric Clothes Dryer                                              63.0
Enhanced Enclosure                                                  24.9
Basic Enclosure                                                     16.4
Heat Pump Clothes Dryer                                             15.8
High Efficiency Heat Pump with Electri

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2511.5
Basic Enclosure + High Efficiency Whole Home Electrification       2507.9
High Efficiency Whole Home Electrification                         2441.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2288.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2284.9
Mininum Efficiency Whole Home Electrification                      2272.5
High Efficiency Heat Pump with Electric Heat Backup                1103.5
Heat Pump Water Heater                                              976.7
Mininum Efficiency Heat Pump with Electric Heat Backup              971.2
Enhanced Enclosure                                                  907.0
Mininum Efficiency Heat Pump with Existing Heat Backup              891.2
Basic Enclosure                                                     835.9
Electric Cooking                                                    196.9
Electric Clothes Dryer   

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1205.5
Basic Enclosure + High Efficiency Whole Home Electrification       1203.6
High Efficiency Whole Home Electrification                         1173.4
Mininum Efficiency Whole Home Electrification                      1144.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1045.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1043.6
High Efficiency Heat Pump with Electric Heat Backup                 428.6
Heat Pump Water Heater                                              405.5
Mininum Efficiency Heat Pump with Electric Heat Backup              361.0
Enhanced Enclosure                                                  351.7
Mininum Efficiency Heat Pump with Existing Heat Backup              323.7
Basic Enclosure                                                     314.7
Electric Cooking                                                    153.3
Induction Cooking        

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [32]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [33]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [34]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [35]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [36]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,3.3,2.5
Basic Enclosure + High Efficiency Whole Home Electrification,3.3,2.5
High Efficiency Whole Home Electrification,3.3,2.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,3.3,2.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,3.3,2.6
Heat Pump Water Heater,3.4,3.0
Enhanced Enclosure,3.3,3.0
Basic Enclosure,3.4,3.1
Mininum Efficiency Whole Home Electrification,3.4,3.1


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,7.9,6.3
Basic Enclosure + High Efficiency Whole Home Electrification,7.9,6.3
High Efficiency Whole Home Electrification,7.9,6.3
Mininum Efficiency Whole Home Electrification,8.2,6.9
Heat Pump Water Heater,7.9,6.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,7.9,6.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,7.9,6.6
Heat Pump Clothes Dryer,15.0,14.4
Induction Cooking,8.1,8.0


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,2.9,2.0
Basic Enclosure + High Efficiency Whole Home Electrification,2.9,2.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,2.9,2.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,2.9,2.1
High Efficiency Whole Home Electrification,2.9,2.1
Enhanced Enclosure,2.9,2.4
Basic Enclosure,2.9,2.4
High Efficiency Heat Pump with Electric Heat Backup,2.9,2.6
Heat Pump Water Heater,2.9,2.6


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1.8,1.3
Basic Enclosure + High Efficiency Whole Home Electrification,1.8,1.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1.8,1.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1.8,1.3
High Efficiency Whole Home Electrification,1.8,1.3
Enhanced Enclosure,1.8,1.6
Basic Enclosure,1.8,1.6
Heat Pump Water Heater,1.8,1.6
Mininum Efficiency Whole Home Electrification,1.8,1.7


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Multifamily with 5+ units, 1940-1979, 0-80% AMI 

Segment 2. Occupied, Multifamily with 5+ units, 1980+, 0-80% AMI 

Segment 3. Occupied, Single family detached 1940-1979, 0-80% AMI 

Segment 4. Occupied, Single family attached 1940-1979, 0-80% AMI 

In [37]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Multi-Family with 5+ Units']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1980s', '1990s', '2000s', '2010s']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_median_income = ['0-30%','30-60%','60-80%']

seg4_building_type = ['Single-Family Attached']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &  
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

## 3.1 Upgrade Cost Per Dwelling Unit 

In [38]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Basic Enclosure                                                     2788.4
Enhanced Enclosure                                                  3759.3
Heat Pump Water Heater                                              3761.0
Mininum Efficiency Heat Pump with Electric Heat Backup              5860.7
Mininum Efficiency Whole Home Electrification                       5911.8
Mininum Efficiency Heat Pump with Existing Heat Backup              7429.5
High Efficiency Heat Pump with Electric Heat Backup                19504.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        24727.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25560.3
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                     1280.2
Enhanced Enclosure                                                  1541.1
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Heat Pump Water Heater                                              3780.9
Mininum Efficiency Whole Home Electrification                       4387.7
Mininum Efficiency Heat Pump with Electric Heat Backup              6762.2
Mininum Efficiency Heat Pump with Existing Heat Backup              7602.8
High Efficiency Heat Pump with Electric Heat Backup                22042.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26584.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     26781.1
High Efficie

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Heat Pump Water Heater                                              4076.1
Mininum Efficiency Heat Pump with Electric Heat Backup             12283.2
Basic Enclosure                                                    12956.8
Mininum Efficiency Whole Home Electrification                      14470.5
Mininum Efficiency Heat Pump with Existing Heat Backup             14578.2
Enhanced Enclosure                                                 16446.3
High Efficiency Heat Pump with Electric Heat Backup                32289.1
High Efficiency Whole Home Electrification                         41387.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        43621.0
Enhanced Enc

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Clothes Dryer                                              1692.2
Electric Cooking                                                    1700.0
Heat Pump Clothes Dryer                                             2507.0
Induction Cooking                                                   2591.2
Heat Pump Water Heater                                              3761.0
Basic Enclosure                                                     6830.3
Enhanced Enclosure                                                  8147.5
Mininum Efficiency Heat Pump with Electric Heat Backup             10421.7
Mininum Efficiency Whole Home Electrification                      12318.8
Mininum Efficiency Heat Pump with Existing Heat Backup             12844.6
High Efficiency Heat Pump with Electric Heat Backup                29586.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        37811.0
High Efficiency Whole Home Electrification                         38445.2
Enhanced Enc

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [39]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Heat Pump Water Heater                                             0.02422
Basic Enclosure                                                    0.02241
Enhanced Enclosure                                                 0.01874
Mininum Efficiency Whole Home Electrification                      0.01482
Heat Pump Clothes Dryer                                            0.01140
Basic Enclosure + High Efficiency Whole Home Electrification       0.00677
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00669
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00667
High Efficiency Whole Home Electrification                         0.00666
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00659
Induction Cooking                                                  0.00320
Electric Cooking                                                   0.00227
High Efficiency Heat Pump with Electric Heat Backup                0.00029
Mininum Effi

segment2


upgrade_name
Heat Pump Water Heater                                             0.04303
Mininum Efficiency Whole Home Electrification                      0.02743
Heat Pump Clothes Dryer                                            0.01054
High Efficiency Whole Home Electrification                         0.00668
Basic Enclosure + High Efficiency Whole Home Electrification       0.00665
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00664
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00661
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00660
Basic Enclosure                                                    0.00642
Enhanced Enclosure                                                 0.00611
Electric Cooking                                                   0.00459
Induction Cooking                                                  0.00403
High Efficiency Heat Pump with Electric Heat Backup               -0.00144
Electric Clo

segment3


upgrade_name
Heat Pump Water Heater                                             0.05408
Basic Enclosure                                                    0.02939
Enhanced Enclosure                                                 0.02477
Mininum Efficiency Whole Home Electrification                      0.01840
Heat Pump Clothes Dryer                                            0.01619
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01550
Basic Enclosure + High Efficiency Whole Home Electrification       0.01507
High Efficiency Whole Home Electrification                         0.01474
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01472
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01439
High Efficiency Heat Pump with Electric Heat Backup                0.00846
Induction Cooking                                                  0.00386
Electric Cooking                                                   0.00265
Mininum Effi

segment4


upgrade_name
Basic Enclosure                                                    0.03353
Enhanced Enclosure                                                 0.03046
Heat Pump Water Heater                                             0.02908
Mininum Efficiency Whole Home Electrification                      0.01818
Heat Pump Clothes Dryer                                            0.01560
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01516
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01488
Basic Enclosure + High Efficiency Whole Home Electrification       0.01466
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01442
High Efficiency Whole Home Electrification                         0.01412
High Efficiency Heat Pump with Electric Heat Backup                0.00855
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00784
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00577
Induction Co

 ### Annual Energy Savings Per Dollar Invested

In [40]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Heat Pump Water Heater                                             0.00182
Mininum Efficiency Whole Home Electrification                      0.00157
Electric Cooking                                                   0.00083
Basic Enclosure                                                    0.00070
Enhanced Enclosure                                                 0.00058
Induction Cooking                                                  0.00049
High Efficiency Whole Home Electrification                         0.00041
Basic Enclosure + High Efficiency Whole Home Electrification       0.00040
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00039
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00039
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00038
Heat Pump Clothes Dryer                                            0.00032
Electric Clothes Dryer                                             0.00025
High Efficie

segment2


upgrade_name
Heat Pump Water Heater                                             0.00214
Mininum Efficiency Whole Home Electrification                      0.00191
Electric Cooking                                                   0.00086
Induction Cooking                                                  0.00042
High Efficiency Whole Home Electrification                         0.00033
Basic Enclosure + High Efficiency Whole Home Electrification       0.00033
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00033
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00032
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00032
Heat Pump Clothes Dryer                                            0.00025
Basic Enclosure                                                    0.00019
Electric Clothes Dryer                                             0.00019
Enhanced Enclosure                                                 0.00018
High Efficie

segment3


upgrade_name
Heat Pump Water Heater                                             0.00329
Mininum Efficiency Whole Home Electrification                      0.00178
Electric Cooking                                                   0.00111
Basic Enclosure                                                    0.00110
Enhanced Enclosure                                                 0.00093
High Efficiency Whole Home Electrification                         0.00080
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00076
Basic Enclosure + High Efficiency Whole Home Electrification       0.00074
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00072
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00070
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00070
Heat Pump Clothes Dryer                                            0.00060
Induction Cooking                                                  0.00060
Mininum Effi

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00162
Heat Pump Water Heater                                             0.00146
Basic Enclosure                                                    0.00130
Enhanced Enclosure                                                 0.00119
Electric Cooking                                                   0.00096
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00075
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00065
High Efficiency Whole Home Electrification                         0.00064
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00063
Basic Enclosure + High Efficiency Whole Home Electrification       0.00062
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00061
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00055
Induction Cooking                                                  0.00049
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [41]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Heat Pump Water Heater                                             0.14488
Mininum Efficiency Whole Home Electrification                      0.14130
Electric Cooking                                                   0.08855
Electric Clothes Dryer                                             0.05578
Induction Cooking                                                  0.04812
Basic Enclosure                                                    0.03350
High Efficiency Whole Home Electrification                         0.03211
Basic Enclosure + High Efficiency Whole Home Electrification       0.03064
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02981
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02946
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02856
Enhanced Enclosure                                                 0.02761
Heat Pump Clothes Dryer                                            0.01172
Mininum Effi

segment2


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.15484
Heat Pump Water Heater                                             0.14403
Electric Cooking                                                   0.09211
Electric Clothes Dryer                                             0.04140
Induction Cooking                                                  0.04035
High Efficiency Whole Home Electrification                         0.02330
Basic Enclosure + High Efficiency Whole Home Electrification       0.02278
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02264
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02179
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02164
Basic Enclosure                                                    0.00840
Enhanced Enclosure                                                 0.00805
Heat Pump Clothes Dryer                                            0.00609
High Efficie

segment3


upgrade_name
Heat Pump Water Heater                                             0.25044
Mininum Efficiency Whole Home Electrification                      0.15830
Electric Cooking                                                   0.11835
Electric Clothes Dryer                                             0.09488
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07578
Basic Enclosure                                                    0.06315
High Efficiency Whole Home Electrification                         0.05922
Mininum Efficiency Heat Pump with Existing Heat Backup             0.05861
Induction Cooking                                                  0.05852
Enhanced Enclosure                                                 0.05339
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.05242
Basic Enclosure + High Efficiency Whole Home Electrification       0.05174
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04948
Enhanced Enc

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.14140
Electric Cooking                                                   0.10516
Heat Pump Water Heater                                             0.10069
Basic Enclosure                                                    0.07685
Enhanced Enclosure                                                 0.07089
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06734
Mininum Efficiency Heat Pump with Existing Heat Backup             0.04994
Electric Clothes Dryer                                             0.04957
Induction Cooking                                                  0.04770
High Efficiency Whole Home Electrification                         0.04321
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04096
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03996
Basic Enclosure + High Efficiency Whole Home Electrification       0.03979
Enhanced Enc

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [42]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1317222.3
Basic Enclosure + High Efficiency Whole Home Electrification       1297974.0
High Efficiency Whole Home Electrification                         1206899.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1146960.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1127712.3
Heat Pump Water Heater                                              609558.0
Mininum Efficiency Whole Home Electrification                       567005.8
Enhanced Enclosure                                                  480078.0
Basic Enclosure                                                     418183.1
Heat Pump Clothes Dryer                                              65362.1
Induction Cooking                                                    56041.8
High Efficiency Heat Pump with Electric Heat Backup                  38219.1
Electric Cooking                                               

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1962260.2
Basic Enclosure + High Efficiency Whole Home Electrification       1951297.4
High Efficiency Whole Home Electrification                         1910640.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1721736.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1710774.1
Heat Pump Water Heater                                             1584874.4
Mininum Efficiency Whole Home Electrification                      1098831.3
Heat Pump Clothes Dryer                                             107071.9
Induction Cooking                                                    97775.9
Enhanced Enclosure                                                   73249.4
Basic Enclosure                                                      64025.4
Electric Cooking                                                     47645.2
Electric Clothes Dryer                                         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    7453864.0
Basic Enclosure + High Efficiency Whole Home Electrification       7400811.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6880001.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6826948.4
High Efficiency Whole Home Electrification                         6159055.0
Enhanced Enclosure                                                 4113297.3
Basic Enclosure                                                    3844894.9
High Efficiency Heat Pump with Electric Heat Backup                2757981.9
Mininum Efficiency Whole Home Electrification                      2623276.9
Heat Pump Water Heater                                             2225248.3
Heat Pump Clothes Dryer                                             403070.3
Mininum Efficiency Heat Pump with Electric Heat Backup              185812.8
Induction Cooking                                              

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    929489.4
Basic Enclosure + High Efficiency Whole Home Electrification       922797.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     847653.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        840960.8
High Efficiency Whole Home Electrification                         796405.9
High Efficiency Heat Pump with Electric Heat Backup                371378.8
Enhanced Enclosure                                                 364110.4
Basic Enclosure                                                    336021.8
Mininum Efficiency Whole Home Electrification                      273795.8
Heat Pump Water Heater                                             160480.3
Mininum Efficiency Heat Pump with Electric Heat Backup             119887.0
Mininum Efficiency Heat Pump with Existing Heat Backup             108790.1
Heat Pump Clothes Dryer                                             57375.2

### Annual Total Energy Savings

In [43]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    77312.1
Basic Enclosure + High Efficiency Whole Home Electrification       76870.4
High Efficiency Whole Home Electrification                         74780.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     66942.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        66500.5
Mininum Efficiency Whole Home Electrification                      60094.8
Heat Pump Water Heater                                             45893.8
Enhanced Enclosure                                                 14834.1
Basic Enclosure                                                    13012.5
Induction Cooking                                                   8528.1
Electric Cooking                                                    7728.6
High Efficiency Heat Pump with Electric Heat Backup                 4827.5
Heat Pump Clothes Dryer                                             1841.8
Electric Clo

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    96478.0
Basic Enclosure + High Efficiency Whole Home Electrification       96226.4
High Efficiency Whole Home Electrification                         95293.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     83782.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        83531.3
Heat Pump Water Heater                                             78687.2
Mininum Efficiency Whole Home Electrification                      76662.2
Induction Cooking                                                  10125.6
Electric Cooking                                                    8975.3
Heat Pump Clothes Dryer                                             2569.5
Enhanced Enclosure                                                  2201.0
Basic Enclosure                                                     1905.5
Electric Clothes Dryer                                                51.4
High Efficie

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    364543.7
Basic Enclosure + High Efficiency Whole Home Electrification       363326.1
High Efficiency Whole Home Electrification                         334828.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     334231.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        333014.3
Mininum Efficiency Whole Home Electrification                      253683.8
Enhanced Enclosure                                                 154356.8
High Efficiency Heat Pump with Electric Heat Backup                145702.8
Basic Enclosure                                                    144071.4
Heat Pump Water Heater                                             135292.1
Mininum Efficiency Heat Pump with Electric Heat Backup              86685.6
Mininum Efficiency Heat Pump with Existing Heat Backup              77057.2
Induction Cooking                                                   15412.4

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    39406.2
Basic Enclosure + High Efficiency Whole Home Electrification       39252.7
High Efficiency Whole Home Electrification                         36352.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     36005.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35851.9
Mininum Efficiency Whole Home Electrification                      24358.4
High Efficiency Heat Pump with Electric Heat Backup                17193.7
Enhanced Enclosure                                                 14260.5
Basic Enclosure                                                    13045.3
Mininum Efficiency Heat Pump with Electric Heat Backup             11423.1
Mininum Efficiency Heat Pump with Existing Heat Backup             10365.8
Heat Pump Water Heater                                              8065.3
Induction Cooking                                                   1871.3
Electric Coo

### Annual Total Emission Savings

In [44]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5885543.0
Basic Enclosure + High Efficiency Whole Home Electrification       5875420.6
High Efficiency Whole Home Electrification                         5822326.4
Mininum Efficiency Whole Home Electrification                      5406183.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4974815.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4964693.1
Heat Pump Water Heater                                             3646539.0
Induction Cooking                                                   843533.3
Electric Cooking                                                    826773.7
Enhanced Enclosure                                                  707376.5
Basic Enclosure                                                     625057.2
High Efficiency Heat Pump with Electric Heat Backup                 549736.2
Mininum Efficiency Heat Pump with Electric Heat Backup         

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    6687699.2
Basic Enclosure + High Efficiency Whole Home Electrification       6684066.8
High Efficiency Whole Home Electrification                         6663327.7
Mininum Efficiency Whole Home Electrification                      6203179.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5645632.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5642000.4
Heat Pump Water Heater                                             5304897.1
Induction Cooking                                                   980202.0
Electric Cooking                                                    956104.0
Enhanced Enclosure                                                   96609.8
Basic Enclosure                                                      83741.4
Heat Pump Clothes Dryer                                              61864.4
Electric Clothes Dryer                                         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    25443091.4
Basic Enclosure + High Efficiency Whole Home Electrification       25410758.5
High Efficiency Whole Home Electrification                         24745510.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     23117982.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        23085649.8
Mininum Efficiency Whole Home Electrification                      22566191.9
High Efficiency Heat Pump with Electric Heat Backup                10735561.9
Heat Pump Water Heater                                             10305526.4
Mininum Efficiency Heat Pump with Electric Heat Backup              9397301.2
Enhanced Enclosure                                                  8864240.9
Mininum Efficiency Heat Pump with Existing Heat Backup              8626515.1
Basic Enclosure                                                     8260037.8
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2509851.2
Basic Enclosure + High Efficiency Whole Home Electrification       2505156.3
High Efficiency Whole Home Electrification                         2437782.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2277143.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2272448.3
Mininum Efficiency Whole Home Electrification                      2130008.3
High Efficiency Heat Pump with Electric Heat Backup                1167744.1
Mininum Efficiency Heat Pump with Electric Heat Backup             1029803.4
Mininum Efficiency Heat Pump with Existing Heat Backup              941238.5
Enhanced Enclosure                                                  847495.5
Basic Enclosure                                                     770235.2
Heat Pump Water Heater                                              555681.4
Induction Cooking                                              

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3926536263.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [45]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    193.3
Basic Enclosure + High Efficiency Whole Home Electrification       190.5
High Efficiency Whole Home Electrification                         177.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     168.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        165.5
Heat Pump Water Heater                                              91.1
Mininum Efficiency Whole Home Electrification                       87.6
Enhanced Enclosure                                                  70.5
Basic Enclosure                                                     62.5
Heat Pump Clothes Dryer                                             28.6
Induction Cooking                                                    8.3
High Efficiency Heat Pump with Electric Heat Backup                  5.6
Electric Cooking                                                     3.9
Electric Clothes Dryer                

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    201.4
Basic Enclosure + High Efficiency Whole Home Electrification       200.3
High Efficiency Whole Home Electrification                         196.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     176.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        175.6
Heat Pump Water Heater                                             162.7
Mininum Efficiency Whole Home Electrification                      120.3
Heat Pump Clothes Dryer                                             26.4
Induction Cooking                                                   10.4
Enhanced Enclosure                                                   9.4
Basic Enclosure                                                      8.2
Electric Cooking                                                     7.8
Electric Clothes Dryer                                             -16.3
High Efficiency Heat Pump with Electri

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    738.3
Basic Enclosure + High Efficiency Whole Home Electrification       733.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     681.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        676.2
High Efficiency Whole Home Electrification                         610.1
Enhanced Enclosure                                                 407.4
Basic Enclosure                                                    380.8
High Efficiency Heat Pump with Electric Heat Backup                273.2
Mininum Efficiency Whole Home Electrification                      266.3
Heat Pump Water Heater                                             220.4
Heat Pump Clothes Dryer                                             40.6
Mininum Efficiency Heat Pump with Electric Heat Backup              18.4
Induction Cooking                                                   10.0
Mininum Efficiency Heat Pump with Exis

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    633.4
Basic Enclosure + High Efficiency Whole Home Electrification       628.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     577.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        573.1
High Efficiency Whole Home Electrification                         542.7
High Efficiency Heat Pump with Electric Heat Backup                253.1
Enhanced Enclosure                                                 248.1
Basic Enclosure                                                    229.0
Mininum Efficiency Whole Home Electrification                      223.9
Heat Pump Water Heater                                             109.4
Mininum Efficiency Heat Pump with Electric Heat Backup              81.7
Mininum Efficiency Heat Pump with Existing Heat Backup              74.1
Heat Pump Clothes Dryer                                             39.1
Induction Cooking                     

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3071005957.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3071005957.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3071005957.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Energy Savings Per Dwelling Unit

In [46]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11.3
Basic Enclosure + High Efficiency Whole Home Electrification       11.3
High Efficiency Whole Home Electrification                         11.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      9.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         9.8
Mininum Efficiency Whole Home Electrification                       9.3
Heat Pump Water Heater                                              6.9
Enhanced Enclosure                                                  2.2
Basic Enclosure                                                     1.9
Electric Cooking                                                    1.4
Induction Cooking                                                   1.3
Heat Pump Clothes Dryer                                             0.8
High Efficiency Heat Pump with Electric Heat Backup                 0.7
Electric Clothes Dryer                             

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    9.9
Basic Enclosure + High Efficiency Whole Home Electrification       9.9
High Efficiency Whole Home Electrification                         9.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     8.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        8.6
Mininum Efficiency Whole Home Electrification                      8.4
Heat Pump Water Heater                                             8.1
Electric Cooking                                                   1.5
Induction Cooking                                                  1.1
Heat Pump Clothes Dryer                                            0.6
Electric Clothes Dryer                                             0.3
Enhanced Enclosure                                                 0.3
Basic Enclosure                                                    0.2
High Efficiency Heat Pump with Electric Heat Backup             

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    36.1
Basic Enclosure + High Efficiency Whole Home Electrification       36.0
High Efficiency Whole Home Electrification                         33.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     33.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        33.0
Mininum Efficiency Whole Home Electrification                      25.8
Enhanced Enclosure                                                 15.3
High Efficiency Heat Pump with Electric Heat Backup                14.4
Basic Enclosure                                                    14.3
Heat Pump Water Heater                                             13.4
Mininum Efficiency Heat Pump with Electric Heat Backup              8.6
Mininum Efficiency Heat Pump with Existing Heat Backup              7.6
Electric Cooking                                                    1.9
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    26.9
Basic Enclosure + High Efficiency Whole Home Electrification       26.8
High Efficiency Whole Home Electrification                         24.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     24.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        24.4
Mininum Efficiency Whole Home Electrification                      19.9
High Efficiency Heat Pump with Electric Heat Backup                11.7
Enhanced Enclosure                                                  9.7
Basic Enclosure                                                     8.9
Mininum Efficiency Heat Pump with Electric Heat Backup              7.8
Mininum Efficiency Heat Pump with Existing Heat Backup              7.1
Heat Pump Water Heater                                              5.5
Electric Cooking                                                    1.6
Induction Cooking                                  

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/396568384.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/396568384.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/396568384.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Annual Percentage Energy Savings Per Dwelling Unit

In [47]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    41.7
Basic Enclosure + High Efficiency Whole Home Electrification       41.5
High Efficiency Whole Home Electrification                         40.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35.5
Mininum Efficiency Whole Home Electrification                      34.0
Heat Pump Water Heater                                             26.2
Enhanced Enclosure                                                  7.8
Basic Enclosure                                                     7.0
Electric Cooking                                                    5.4
Induction Cooking                                                   4.8
Heat Pump Clothes Dryer                                             3.4
Electric Clothes Dryer                                              1.7
High Efficiency Heat Pump with Electric Heat Backup

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    35.9
Basic Enclosure + High Efficiency Whole Home Electrification       35.8
High Efficiency Whole Home Electrification                         35.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30.5
Mininum Efficiency Whole Home Electrification                      30.2
Heat Pump Water Heater                                             30.0
Electric Cooking                                                    6.0
Induction Cooking                                                   4.4
Heat Pump Clothes Dryer                                             2.4
Enhanced Enclosure                                                  1.1
Electric Clothes Dryer                                              1.1
Basic Enclosure                                                     1.0
High Efficiency Heat Pump with Electric Heat Backup

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    49.3
Basic Enclosure + High Efficiency Whole Home Electrification       49.1
High Efficiency Whole Home Electrification                         45.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     44.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44.2
Mininum Efficiency Whole Home Electrification                      33.7
Heat Pump Water Heater                                             21.0
Enhanced Enclosure                                                 19.7
Basic Enclosure                                                    18.2
High Efficiency Heat Pump with Electric Heat Backup                15.4
Mininum Efficiency Heat Pump with Electric Heat Backup              6.1
Mininum Efficiency Heat Pump with Existing Heat Backup              4.8
Electric Cooking                                                    3.1
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    51.7
Basic Enclosure + High Efficiency Whole Home Electrification       51.5
High Efficiency Whole Home Electrification                         47.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     46.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        46.3
Mininum Efficiency Whole Home Electrification                      33.7
High Efficiency Heat Pump with Electric Heat Backup                20.3
Enhanced Enclosure                                                 16.9
Basic Enclosure                                                    15.8
Heat Pump Water Heater                                             13.0
Mininum Efficiency Heat Pump with Electric Heat Backup             12.2
Mininum Efficiency Heat Pump with Existing Heat Backup             11.1
Electric Cooking                                                    3.3
Induction Cooking                                  

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/1358844843.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/1358844843.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/1358844843.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Emission Savings Per Dwelling Unit

In [48]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    863.7
Basic Enclosure + High Efficiency Whole Home Electrification       862.2
High Efficiency Whole Home Electrification                         854.4
Mininum Efficiency Whole Home Electrification                      835.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     730.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        728.6
Heat Pump Water Heater                                             544.9
Electric Cooking                                                   150.5
Induction Cooking                                                  124.7
Enhanced Enclosure                                                 103.8
Electric Clothes Dryer                                              94.4
Basic Enclosure                                                     93.4
High Efficiency Heat Pump with Electric Heat Backup                 80.7
Mininum Efficiency Heat Pump with Elec

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    686.5
Basic Enclosure + High Efficiency Whole Home Electrification       686.1
High Efficiency Whole Home Electrification                         684.0
Mininum Efficiency Whole Home Electrification                      679.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     579.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        579.2
Heat Pump Water Heater                                             544.6
Electric Cooking                                                   156.6
Induction Cooking                                                  104.6
Electric Clothes Dryer                                              70.1
Heat Pump Clothes Dryer                                             15.3
Enhanced Enclosure                                                  12.4
Basic Enclosure                                                     10.8
High Efficiency Heat Pump with Electri

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2520.2
Basic Enclosure + High Efficiency Whole Home Electrification       2517.0
High Efficiency Whole Home Electrification                         2451.1
Mininum Efficiency Whole Home Electrification                      2290.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2289.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2286.7
High Efficiency Heat Pump with Electric Heat Backup                1063.4
Heat Pump Water Heater                                             1020.8
Mininum Efficiency Heat Pump with Electric Heat Backup              930.8
Enhanced Enclosure                                                  878.0
Mininum Efficiency Heat Pump with Existing Heat Backup              854.5
Basic Enclosure                                                     818.2
Electric Cooking                                                    201.2
Electric Clothes Dryer   

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      1741.9
Enhanced Enclosure + High Efficiency Whole Home Electrification    1710.4
Basic Enclosure + High Efficiency Whole Home Electrification       1707.2
High Efficiency Whole Home Electrification                         1661.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1551.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1548.7
High Efficiency Heat Pump with Electric Heat Backup                 795.8
Mininum Efficiency Heat Pump with Electric Heat Backup              701.8
Mininum Efficiency Heat Pump with Existing Heat Backup              641.4
Enhanced Enclosure                                                  577.6
Basic Enclosure                                                     524.9
Heat Pump Water Heater                                              378.7
Electric Cooking                                                    178.8
Induction Cooking        

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/624873452.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/624873452.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/624873452.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [49]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Segment 2

In [50]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 3

In [51]:
egment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/1326448381.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/1326448381.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/1326448381.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 4

In [52]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/2289506022.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/2289506022.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/2289506022.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [53]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,4.2,3.1
Basic Enclosure + High Efficiency Whole Home Electrification,4.2,3.1
High Efficiency Whole Home Electrification,4.2,3.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,4.2,3.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,4.2,3.3
Heat Pump Water Heater,4.2,3.8
Enhanced Enclosure,4.2,3.7
Basic Enclosure,4.2,3.8
Mininum Efficiency Whole Home Electrification,4.3,3.9


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,9.7,7.7
Basic Enclosure + High Efficiency Whole Home Electrification,9.7,7.7
High Efficiency Whole Home Electrification,9.7,7.7
Mininum Efficiency Whole Home Electrification,10.1,8.5
Heat Pump Water Heater,9.7,8.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,9.7,8.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,9.7,8.1
Heat Pump Clothes Dryer,16.9,16.2
Induction Cooking,9.9,9.8


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,4.1,2.8
Basic Enclosure + High Efficiency Whole Home Electrification,4.1,2.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,4.1,2.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,4.1,2.9
High Efficiency Whole Home Electrification,4.1,3.0
Enhanced Enclosure,4.1,3.3
Basic Enclosure,4.1,3.4
Heat Pump Water Heater,4.1,3.7
High Efficiency Heat Pump with Electric Heat Backup,4.1,3.7


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,2.9,2.0
Basic Enclosure + High Efficiency Whole Home Electrification,2.9,2.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,2.9,2.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,2.9,2.1
High Efficiency Whole Home Electrification,2.9,2.1
Enhanced Enclosure,2.9,2.6
Basic Enclosure,2.9,2.6
Heat Pump Water Heater,2.9,2.7
Mininum Efficiency Whole Home Electrification,3.2,2.9


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_53214/2330438264.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [54]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [55]:
! jupyter nbconvert --output-dir='./results/san_jose/' --to=html --no-input saving_results_reports_for_san_jose.ipynb

[NbConvertApp] Converting notebook saving_results_reports_for_san_jose.ipynb to html
/Users/lliu2/micromamba/envs/res-est/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'application/javascript' -%}
[NbConvertApp] Writing 505085 bytes to results/san_jose/saving_results_reports_for_san_jose.html
